In [7]:
import torch
import torch.nn as nn
from torchsummary import summary
from torchvision import models

import sys, os
sys.path.append(os.path.abspath("../"))

%load_ext autoreload
%autoreload 2

from models.res_u_net.model import ResUnetAutoencoder
from models.res_u_net.std_blocks import UpBlock


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
resnet = models.resnet18(pretrained=True)

In [9]:
bridge_out_channels = 1024

In [15]:
autoencoder = ResUnetAutoencoder(
    resnet,
    bridge_out_channels,
    UpBlock.UPSAMPLING_BILINEAR,
    (232, 232)
)

In [16]:
x = torch.rand(1, 3, 232, 232)

In [17]:
autoencoder

ResUnetAutoencoder(
  (encoder): ResnetEncoder(
    (input_block): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (conv_blocks): ModuleDict(
      (64;64): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

In [18]:
output = autoencoder(x)

In [19]:
output.shape


torch.Size([1, 3, 232, 232])